## GIS 5572: Lab 1
#### By Zain Ul Abdin Siyal (12-Feb-2024)

### Data Pipeline Part

#### Install relevant libraries

In [41]:
!pip install psycopg2-binary

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/cb/35/009b43d67a7010b789690a46d49521fd435ce33ce722fe8d7ac7efe35c21/psycopg2_binary-2.9.9-cp39-cp39-win_amd64.whl.metadata
   ---------------------------------------- 1.2/1.2 MB 5.3 MB/s eta 0:00:00


In [2]:
## Import All Libraries

# for ArcGIS geoprocessing
import arcpy

# for interacting with PostgreSQL/PostGIS database
import psycopg2

# for encoding and decoding JSON data
import json

# for making HTTP requests to web services and APIs
import requests

# for pattern matching and string manipulation
import re #regular expression

# for interacting with the operating system
import os

# arcgis.gis is part of the ArcGIS API for Python, used for working with GIS data and services
from arcgis.gis import GIS

# for interacting with the Python runtime environment
import sys

## Workflow

### Created a Polygon using arcpy Geometry Primitives

In [65]:
# Define the polygon using arcpy geometry primitives
polygon = arcpy.Polygon(
    arcpy.Array([
        arcpy.Point(-93.22456694058349, 44.97656707355932), #Huntington Bank
        arcpy.Point(-93.25750208433475, 44.97369530667097), #US Bank
        arcpy.Point(-93.21775917100368, 44.884844671870496), #MSP Airport
        arcpy.Point(-93.09021376922068, 44.95341865709891), #Downtown St. Paul
    ])
)

### Convert the polygon to Well-Known Text (WKT)

In [66]:
polygon_wkt = polygon.WKT

### Connect to the PostGIS database

In [76]:
conn = psycopg2.connect(
    host="34.122.229.143",
    port="5432",
    database="lab1",
    user="postgres",
    password="your_password"
)

### Create a cursor to execute SQL queries

In [77]:
cur = conn.cursor() 

#### SKIP THE FOLLOWING CELL

In [75]:
### SKIP THIS!
#This will end the current transaction and allow to start a new one

conn.rollback()

InterfaceError: connection already closed

### Execute SQL query to insert the polygon into the PostGIS database

Created Zain_Table in th PGAdmin before running the following query

In [78]:
sql_query = f"INSERT INTO Zain_Table1 (geom) VALUES (ST_GeomFromText('{polygon_wkt}', 4326));"

In [79]:
cur.execute(sql_query) #Execute the above sql_query

### Commit the transaction and close the connection

In [80]:
conn.commit() #saves any pending transactions to the database
cur.close() #closes the cursor associated with the database connection.
conn.close() #closes the database connection 

### Fetching and Parsing Geometric Data from API

In [17]:
# Your API endpoint
api = 'http://34.67.91.226:5000/get_polygon'

try:
    response = requests.get(api)
    response.raise_for_status()
    dictionary = json.loads(response.text)
except requests.exceptions.RequestException as e:
    print(f"Error during API request: {e}")
    exit()

### Processing API Response
A conditional check and data extraction process to handle the geometry information from the API response.

In [18]:
# Check if 'features' key exists
if 'features' in dictionary and len(dictionary['features']) > 0:
    # Extract geometry from the first feature
    geometry_str = dictionary['features'][0]['geometry']

    # Replace multiple occurrences using a regular expression
    geometry_str = re.sub(r'\[\[\[\[\[', '[[[', geometry_str)
    geometry_str = re.sub(r'\]\]\]\]\]', ']]]', geometry_str)

    try:
        # Parse the cleaned geometry string into a dictionary
        geometry_dict = json.loads(geometry_str)
        print(geometry_dict)
    except json.JSONDecodeError as e:
        print(f"Error decoding geometry string: {e}")
else:
    print("Error: 'features' key not found or empty in the API response.")

{'type': 'MultiPolygon', 'coordinates': [[[[-93.224487305, 44.97668457], [-93.257507324, 44.973693848], [-93.217712402, 44.884887695], [-93.090209961, 44.953491211], [-93.224487305, 44.97668457]]]]}


### Create a GeoJSON dictionary

In [19]:
# Specify GeoJSON dictionary name and then create
Lab1geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": geometry_dict['coordinates'],
        "type": "Polygon"
      }
    }
  ]
}

# Use raw string (r'') for Windows paths to handle backslashes
path = os.path.join(r'F:\1. UMN MGIS\1. Semesters\4th Semester\1. ArcGIS II\2. Labs\ArcPro\ArcGIS II', 'Lab1geojson.json')

# Use 'with' statement for file handling to ensure proper closing
with open(path, 'w') as json_file:
    json.dump(Lab1geojson, json_file, indent=2)  # Indent for better readability

### Sign into ArcGIS Online Account

In [16]:
username = 'your_email'
password = 'your_password'

try:
    gis = GIS('https://www.arcgis.com', username, password)
    print("Successfully signed in to ArcGIS Online.")
except Exception as e:
    print(f"Error signing in: {e}")
    # Handle the error or exit the program
    sys.exit()

Error signing in: A general error occurred: Too many invalid attempts. Please try again later.


SystemExit: 